# Конвертирует разметку из json-файла в набор xml-файлов для обучения darkflow

In [1]:
img_shape = (1280,720,3) #размер изображения

In [2]:
import json, os

In [3]:
from dict2xml import dict2xml

In [4]:
json_source_path = 'C:\\my_cv_work\\tpe_recognition\\data\\cam_1_hand_markup'
json_file = 'cam_1_hand_markup_next.json'
images_path = 'D:\\data\\pics\\cam_1_src'

In [5]:
# папка для хранения xml-разметки
xml_path = os.path.join(json_source_path, 'to_xml_boundary_2')
if not os.path.exists(xml_path):
    os.mkdir(xml_path)

In [6]:
# загружаем данные из словаря с разметкой
data = json.loads(open(os.path.join(json_source_path,json_file), 'r').read())

In [7]:
def check_scaled_box(bbox, scale, img_shape):
    x_max, y_max, _ = img_shape
    x_min, y_min = 0., 0.
    shift_x = (scale-1.)*x_max/2.
    shift_y = (scale - 1.)*y_max/2.
    x_b_min, y_b_min = bbox['topleft']
    x_b_max, y_b_max = bbox['bottomright']
    x_b_min, x_b_max = scale*x_b_min - shift_x, scale*x_b_max - shift_x
    y_b_min, y_b_max = scale*y_b_min - shift_y, scale*y_b_max - shift_y
    return (x_b_min*y_b_min > 0. and (x_b_max-x_max)*(y_b_max-y_max)>0.)

In [8]:
def convert_dict_to_xml_style(img_filename, bboxes, folder='folder_1', size=None):
    objects = []
    for bbox in bboxes['bboxes']:
        obj_dict = {}
        obj_dict['name'] = 'person'
        obj_dict['pose'] = 'Left'
        obj_dict['truncated'] = 0
        obj_dict['difficult'] = 0
        obj_dict['bndbox'] = {'xmin': bbox['topleft'][0],
                              'ymin': bbox['topleft'][1],
                              'xmax': bbox['bottomright'][0],
                              'ymax': bbox['bottomright'][1]}
        objects.append(obj_dict)
    sizes = {'width': size[0], 'height': size[1], 'depth': size[2]}
    data = {}
    data['folder'] = folder
    data['filename'] = img_filename
    data['size'] = sizes
    data['object'] = objects
    return {'annotation': data}

In [10]:
# делаем словарь "чистой" разметки:
ground_truth = { k:v for k,v in data.items() if v['n_boxes'] > 0}
# два огранчения: 1) ненулевая площадь ббокса 2) ббокс не расположен на границе изображения (проблемы при аугментации)
# отступы от краев изображений
scale = 1.1
clean_dict = dict()
for k,v in ground_truth.items():
    good_boxes = []
    for bbox in v['bboxes']:
        if (bbox['bottomright'][0]-bbox['topleft'][0])*(bbox['bottomright'][1]-bbox['topleft'][1])>0 and check_scaled_box(bbox, scale, img_shape):
            good_boxes.append(bbox)
    if len(good_boxes)>0:
        clean_dict[k] = {'bboxes': good_boxes}
        
for k,v in clean_dict.items():
    for bbox in v['bboxes']:
        assert (bbox['bottomright'][0]-bbox['topleft'][0])*(bbox['bottomright'][1]-bbox['topleft'][1])>0, 'file: {}'.format(k)
        
print(len(clean_dict))

1203


In [11]:
# делаем xml-файлы
for k,v in clean_dict.items():
    img_filename = os.path.split(k)[-1]
    xml_filename = os.path.splitext(img_filename)[0]+'.xml'
    styled_dict = convert_dict_to_xml_style(img_filename, v, size=img_shape)
    xml_data = dict2xml(styled_dict)
    with open(os.path.join(xml_path, xml_filename), 'w') as f:
        f.write(xml_data)

In [12]:
# случайный выбор файлов train/test
train_part = 0.8
RAND = 1234


In [13]:
import numpy as np

In [16]:
np.random.seed(RAND)

In [29]:
flist = [x for x in os.listdir(xml_path) if os.path.splitext(x)[1] == '.xml']
n_train = int(len(flist)*train_part)
ftrain = np.random.choice(flist, n_train, replace=False)
ftest = [x for x in flist if x not in ftrain]

In [30]:
len(ftest)/len(flist)

0.20031116297160637

In [31]:
train_folder = '../data/cam_1_hand_markup/to_xml_boundary_2/train'
test_folder = '../data/cam_1_hand_markup/to_xml_boundary_2/test'

In [32]:
from shutil import copyfile

In [33]:
for file in ftrain:
    src = os.path.join(xml_path, file)
    dst = os.path.join(train_folder, file)
    copyfile(src, dst)

In [34]:
for file in ftest:
    src = os.path.join(xml_path, file)
    dst = os.path.join(test_folder, file)
    copyfile(src, dst)